# LOFAR ionospheric Calibration

We will discuss here how to obtain ionospheric information from the Lofar interferometry calibration solutions. Some information on Lofar calibration can be found in the cookbook: https://support.astron.nl/LOFARImagingCookbook/. In particular we will be using the solutions obtained with DP3: https://dp3.readthedocs.io/en/latest/.
Because of the wide field, Lofar calibration is done in multiple steps. An initial step tries to remove the global instrumental and other corruptions, the so called Direction Independent (DI) calibration. After that, Direction Dependent (DD) calibration is applied to correct for field of view variations.

### Question: 

In which step do you solve for ionospheric effects?

### 
In this excercise we will only look at direction independent calibration. Most of the ionospheric information can be obtained from those solutions. As we wil learn later, the direction independent solutions give access to ionospheric variations over the array. Direction dependent solutions probe variations over the field of view. 

The main ionospheric effect in your data is a dispersive delay, resulting in a frequency dependent phase error. But the second order effect, differential Faraday rotation should also be taken into account. The main instumental effects can be modeled with a complex gain, independent for X and Y polarisation

$\begin{pmatrix} G_{xx} & G_{xy} \\ G_{yx} & G_{yy} \end{pmatrix} = \begin{pmatrix} cos(\alpha) & sin(\alpha) \\ -sin(\alpha) & cos(\alpha)\end{pmatrix}
\cdot \begin{pmatrix} G_{xx} & 0 \\ 0 & G_{yy} \end{pmatrix} 
$

### Excercise:

Use from DP3 either gaincal or DDECAL to create a parset to use for calibration. Hint: check the "caltype" or "mode" field.

### Question :

You can set the number of solutions per time/frequency interval with solint/nchan parameters. What would you chose for the time and frequency resolutions? Why?

### 
For calibration you need a good source model. Errors in your model will influence the quality of your solutions. If you are interested in the ionospheric content of the calibration solutions, it is best to look at data from a calibrator. During this excercise we will use data from the 3C196 field. 

### Excercise: 

What kind of source is 3C196? What are its coordinates? You can for example use the Simbad source catalog.

### 
After calibration the solutions are stored in hdf5 files. (h5parm). One example solution file for a single subband can be found in "data/solutions.h5". A tool to inspect and post process the solutions is losoto https://support.astron.nl/LOFARImagingCookbook/losoto.html. We will not use losoto now, but it is recommended to use it if you start to work with solution files yourself. You can also directly access the data in python. 

### Excercise:

Open the solutions.h5 file in python using h5py. hdf5 has a tree like structure. You can inspect the structure of the file by iterating through the keys(). Try to plot some of the solutions. Can you plot phases versus time? What about rotation angle. Hint: Since interferometric measurement are sensitive to differential delays between stations, you have to chose a reference station.

In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import ConciseDateFormatter, date2num

In [ ]:
myfile = h5py.File("data/solutions.h5")

In [ ]:
print(myfile.keys())

In [ ]:
print(myfile['sol000'].keys())

In [ ]:
print(myfile['sol000']['phase000'].keys())

In [ ]:
phases = myfile['sol000']['phase000']['val'][:]
phases = phases[:,:,0,0,:] - phases[:,0,0,0,:][:,np.newaxis]
times = myfile['sol000']['phase000']['time'][:]

The time format is mjd in seconds

In [ ]:
from astropy.time import Time
times = Time(times/(3600.*24.),format="mjd")

In [ ]:
fig,ax = plt.subplots()
ax.plot(times.datetime,phases[:,50])
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))

In [ ]:
alpha = myfile['sol000']['rotation000']['val'][:]
alpha = alpha[:,:,0,0] - alpha[:,0,0,0][:,np.newaxis]
times = myfile['sol000']['rotation000']['time'][:]
times = Time(times/(3600.*24.),format="mjd")

In [ ]:
fig,ax = plt.subplots()
ax.plot(times.datetime,alpha[:,55])
ax.xaxis.set_major_formatter(ConciseDateFormatter(ax.xaxis.get_major_locator()))
ax.set_ylim(-0.1,0.1)

### 
The main contribution to the calibration phases come from station clock errors and ionospheric delays. They both have a different frequency dependence. You can use the large bandwidth of Lofar to separate ionospheric phases from clock errors. We do that by fitting for clock and TEC the following function on the phases ($\phi$):

$\phi(\nu) = 2\pi\cdot clock\cdot\nu  + 8.45e^9\cdot TEC/\nu$

With TEC the differential integrated electron density along the line of sight in TECU ($10^{16} e^-/m^2$)

In losoto there is an operation $clocktec$ that does this fit for you, provided you have a large enough bandwidth. So multiples of the solution file above for different frequencies (subbands). 

We have already done the clock tec separation for you. You can now continue with the next notebook to inspect the tec solutions.